# Mismatch detection
>Antonio Colás Nieto, @acolasn, anconi.1999@gmail.com

We will run standard synchronization code and attempt to find and analyse mismatch responses.  

Taken from `demo_analysis.ipynb`, and hoping that I need to change as little as possible. 

In [ ]:
%reload_ext autoreload
%autoreload 2

import functools

print = functools.partial(print, flush=True)

import os
import random
import numpy as np
import pandas as pd
from scipy.stats import zscore, mannwhitneyu, pearsonr

import matplotlib.pyplot as plt
from pathlib import Path
import pickle
from cottage_analysis.analysis import (
    mismatch)
import flexiznam as flz
from tqdm import tqdm

from matplotlib_venn import venn3





In [ ]:
PROJECT = "663214d08993fd0b6e6b5f1d"
PROTOCOL = "KellerTube"

MESSAGES = "harpmessage.bin"

## Main entry point:

In [ ]:
mismatch.analyse_session("BRAC9057.4j_S20240517")

## Building pipeline

In [ ]:
# Example session
mouse = "BRAC9057.4j"
session = "BRAC9057.4j_S20240516"

flexilims_session = flz.get_flexilims_session(project_id=PROJECT)
# all_protocol_recording_entries = generate_filepaths.get_all_recording_entries(project=project,
#                                                                               mouse=mouse,
#                                                                               session=session,
#                                                                               protocol=protocol,
#                                                                               flexilims_session=flexilims_session)

# # DO NOT RUN THIS FUNCTION (TAKES 2hrs ish): to find monitor frames from photodiode signal
# find_monitor_frames(project=project,
#                     mouse=mouse,
#                     session=session,
#                     protocol=protocol,
#                     all_protocol_recording_entries=None,
#                     irecording=None,
#                     flexilims_session=None)


exp_session = flz.get_entity(
    datatype="session", name=session, flexilims_session=flexilims_session
)

si_datasets = flz.get_datasets(
    exp_session['id'],
    dataset_type='suite2p_rois',
    flexilims_session=flexilims_session
)

suite2p_dataset = si_datasets[0]

In [ ]:
recordings = flz.get_entities(
    datatype="recording",
    origin_id=exp_session['id'],
    query_key="recording_type",
    query_value="two_photon",
    flexilims_session=flexilims_session,
)

In [ ]:
recordings.name[1]

In [ ]:
which_recording= 3

recname = recordings.name[which_recording]

recording = flz.get_entity(
    datatype="recording",
    name=recname,
    flexilims_session=flexilims_session,
)

In [ ]:
recording.name

## Synchronization

Ideally, this would work out of the bat. 

In [ ]:
# Generate synchronisation dataframes

vs_df_all, imaging_df_all, recordings = mismatch.sync_all_recordings(
    session_name=session,
    flexilims_session=flexilims_session,
    project=PROJECT,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="KellerTube",
    photodiode_protocol=5,
    return_volumes=True,
)

# Analysis: have a working pipeline

Assuming there is only one  depth recording

In [ ]:
closed_loop = imaging_df_all[which_recording-1]

In [ ]:
#plt.plot(imaging_closed_loop_all["mouse_z"]/imaging_closed_loop_all["mismatch_mouse_z"])
plt.plot(closed_loop["mouse_z"])


## Finding mismatches

In [ ]:
closed_loop["mousez_dif"]  = np.zeros(len(closed_loop["mouse_z"]))
closed_loop.loc[1:, 'mousez_dif'] = np.diff(closed_loop['mouse_z'])


In [ ]:
closed_loop["mismz_dif"]  = np.zeros(len(closed_loop["mouse_z"]))
closed_loop.loc[1:, 'mismz_dif'] = np.diff(closed_loop['mismatch_mouse_z'])

In [ ]:
closed_loop["mism_ratio"] = closed_loop["mousez_dif"]/closed_loop["mismz_dif"]

In [ ]:
plt.plot(closed_loop["mism_ratio"][720:740])

In [ ]:
closed_loop["mismatch"] = ((closed_loop["mism_ratio"] > 1.2) | (closed_loop["mism_ratio"] < -1000)).astype(int)

#To catch the fact that it's -inf sometimes during a mismatch

In [ ]:
up= 0
down =  -1

plt.plot(closed_loop["mism_ratio"][up:down])
plt.plot(closed_loop["mismatch"][up:down])

closed_loop["mism_ratio"][up:down]

In [ ]:
closed_loop.drop(columns = {"mismz_dif", "mousez_dif"})

## Defining mismatch window for raster

By design, 5 frames before and 10 after the mismatch onset frame. The length is 15 frames. 

In [ ]:
# Create a new column initialized to 0
closed_loop['range_indicator'] = 0

#Make a diff to look at starting frames
closed_loop["start_mismatch"]=np.zeros(len(closed_loop["mismatch"]))

closed_loop.loc[1:,"start_mismatch"] = np.diff(closed_loop["mismatch"])

# Find indices where 'indicator' is 1
indices = closed_loop.index[closed_loop['start_mismatch'] == 1].tolist()
print(indices)

# Set range_indicator to 1 for 5 rows before and after each index where 'indicator' is 1
for idx in indices:
    start = max(idx - 5, 0)
    end = min(idx + 10, len(closed_loop['mismatch']) - 1)
    print((start,  end))
    closed_loop.loc[start:end, 'range_indicator'] = 1

For every neuron, we want to find a point in time where the mismatch begins and calculate the average of its responses to a mismatch

In [ ]:
# Create the initial DataFrame with range_indicator
neurons_closed_loop = pd.DataFrame({"range_indicator": closed_loop["range_indicator"].copy()})

# Extract the number of neurons
neurons = closed_loop["closed_loopfs"][0].shape[1]

# Create a DataFrame for the neurons data
neuron_data = pd.DataFrame(
    {f"neuron{neuron}": [closed_loop["closed_loopfs"][i][0][neuron] for i in range(len(closed_loop["mismatch"]))]
     for neuron in range(neurons)}
)

# Concatenate the range_indicator and neuron data
neurons_closed_loop = pd.concat([neurons_closed_loop, neuron_data], axis=1)

In [ ]:
mismatches_per_neuron = list(np.zeros(neurons))

neurons_closed_loop["start_mismatch"]=np.zeros(len(neurons_closed_loop["range_indicator"]))

neurons_df.loc[1:,"start_mismatch"] = np.diff(neurons_df["range_indicator"])

n_mismatches  =  len(neurons_df["start_mismatch"][neurons_df["start_mismatch"]==1])

print(n_mismatches)

for i in range(neurons):
    mismatches_per_neuron[i] = np.zeros((n_mismatches, 15))



In [ ]:


# Initialize variables to track the start and end of intervals
in_interval = False
start_idx = None

# Iterate through the DataFrame to identify intervals
idx_mismatch = -1
for idx, row in neurons_df.iterrows():
    if row['range_indicator'] == 1 and not in_interval:
        # Start of a new interval
        start_idx = idx
        in_interval = True
        idx_mismatch += 1
        print(f"This is mismatch {idx_mismatch}")
    elif row['range_indicator'] == 0 and in_interval:
        # End of the current interval
        end_idx = idx-1
        for neuron in range(neurons):
            mismatches_per_neuron[neuron][idx_mismatch, :] = neurons_df[f"neuron{neuron}"][start_idx:end_idx]
        in_interval = False
        print(f"start and end idx: {(start_idx, end_idx)}")

In [ ]:
mismatch_raster = np.zeros((neurons, 15))

for i in range(neurons):
    mismatch_raster[i, :] = np.mean(mismatches_per_neuron[i], axis = 0)

#print(mismatch_raster.shape)

# Define a function to calculate the difference for each row
def calculate_difference(row):
    first_5_sum = np.sum(row[0:5])
    last_5_sum = np.sum(row[6:11])
    return last_5_sum-first_5_sum

# Calculate differences for each row
differences = np.apply_along_axis(calculate_difference, 1, mismatch_raster)
#print(differences[0:10])

# Get the sorted indices based on the differences (larger differences first)
sorted_indices = np.argsort(-differences)
#print(sorted_indices[0:10])

# Sort the array based on the calculated differences
sorted_mismatch_raster = mismatch_raster[sorted_indices]

start = 0
end = 100

fig = plt.figure(figsize=(30,10),facecolor='w') 
ax = fig.add_subplot(111)
im = ax.imshow(sorted_mismatch_raster[0:100])



ax.set_title(f"Raster plot of first {end} neurons aligned to mismatch")
ax.set_xlabel("Frames")
ax.set_ylabel("Neurons")
fig.colorbar(im, label  =  "dff")
ax.axvline(5, color = "grey")

In [ ]:
plt.plot(differences[sorted_indices])


In [ ]:
pop_response  = np.mean(sorted_mismatch_raster[0:100], axis = 0)
plt.plot(pop_response)

## Now sorting a la Keller

Evaluate significant mismatch modulation  by producing a series of  random events and looking at the differential modullation from mismatches. 

In [ ]:
def generate_random_events(n_frames, n_events = 100):
    return [random.randint(0, n_frames-1) for _ in range(n_events)]

In [ ]:
n_frames = len(closed_loop)
events = generate_random_events(n_frames)

In [ ]:
closed_loop["randevents"] = 0
closed_loop.loc[events, "randevents"] = 1
plt.plot(closed_loop["randevents"])

In [ ]:
rand_rec, indices = mismatch.create_mismatch_window(closed_loop, window_start = 0, window_end = 5, event  = "randevents")

In [ ]:
neurons, rand_neurodf = mismatch.build_neurons_df(rand_rec)

In [ ]:
rand_misperneuron = mismatch.build_mismatches_per_neuron_list(neurons, rand_neurodf, window_start = 0, window_end = 5, indices = indices)

In [ ]:
rand_raster = mismatch.raster(neurons, rand_misperneuron, window_start = 0, window_end= 5)

In [ ]:
rand_avg = np.mean(rand_raster, axis = 1)
mismatch_avg = np.mean(mismatch_raster[:, 8:13], axis = 1)
modulation_raster = mismatch_avg-rand_avg

In [ ]:
sorted_indices = np.argsort(-modulation_raster)
#print(sorted_indices[0:10])

plt.plot(modulation_raster[sorted_indices])

In [ ]:


# Sort the array based on the calculated differences
sorted_mismatch_raster = mismatch_raster[sorted_indices]

# Define the range you want to cap
vmin = -0.5
vmax = 1

start = 0
end = 100
fig = plt.figure(figsize=(30, 10), facecolor='w')
ax = fig.add_subplot(111)
im = ax.imshow(sorted_mismatch_raster, cmap='coolwarm', vmin=vmin, vmax=vmax, aspect =  "auto")


ax.set_title(f"Raster plot of neurons aligned to mismatch")
ax.set_xlabel("Frames")
ax.set_ylabel("Neurons")
fig.colorbar(im, label="dff")
ax.axvline(5, color="grey")
plt.show()

In [ ]:
pop_response  = np.mean(sorted_mismatch_raster, axis = 0)
plt.plot(pop_response)

In [ ]:
closed_loop, indices = mismatch.create_mismatch_window(closed_loop, window_start = 5, window_end = 20)
neurons, neurons_df = mismatch.build_neurons_df(closed_loop)
misperneuron = mismatch.build_mismatches_per_neuron_list(neurons, neurons_df, window_start = 5, window_end = 20)
mismatch_raster = mismatch.raster(neurons, misperneuron, window_start = 5, window_end = 20)
rand_raster = mismatch.make_rand_raster(closed_loop, n_events = 200, window_end = 10)
sorted_mismatch_raster = mismatch.modulation_sort_raster(rand_raster, mismatch_raster)
mismatch.plot_raster(sorted_mismatch_raster, vmin = -0.5,  vmax = 0.8)

In [ ]:


mismatch.plot_raster(sorted_mismatch_raster, vmin = -0.5,  vmax = 0.8)

## Z-scoring and trying new preprocess pipeline

In [ ]:
closed_loop = mismatch.find_mismatch(closed_loop)

closed_loop, indices = mismatch.create_mismatch_window(closed_loop, window_start = 5, window_end = 20)
neurons, neurons_df = mismatch.build_neurons_df(closed_loop)


In [ ]:
misperneuron = mismatch.build_mismatches_per_neuron_list(neurons, neurons_df, window_start = 5, window_end = 20)
mismatch_raster = mismatch.raster(neurons, misperneuron, window_start = 5, window_end = 20)

In [ ]:
rand_raster = mismatch.make_rand_raster(closed_loop, n_events = 200, window_end = 10)

In [ ]:
sorted_mismatch_raster, modulation_raster = mismatch.modulation_sort_raster(rand_raster, mismatch_raster)
mismatch.plot_raster(sorted_mismatch_raster, vmin = -1,  vmax = 1)


In [ ]:
plt.plot(sorted_mismatch_raster[-10:,:].T)

In [ ]:
plt.plot(modulation_raster[np.argsort(-modulation_raster)])
plt.axhline(0, color="red")
plt.title("Z-scores sorted from high to low")
plt.ylabel("Z-score")
plt.xlabel("Neurons")

In [ ]:
def plot_pop_response(sorted_mismatch_raster, how_many=None):
    fig, ax = plt.subplots()

    ax.plot(np.mean(sorted_mismatch_raster, axis=0), label="Total")

    if how_many is None:
        how_many = int(0.1 * len(sorted_mismatch_raster))

    ax.plot(np.mean(sorted_mismatch_raster[:how_many, :], axis=0), label=f"First {how_many}")
    ax.plot(np.mean(sorted_mismatch_raster[-how_many:, :], axis=0), label=f"Bottom {how_many}")

    ax.set_title("Population response")
    ax.set_ylabel("Z-score")
    ax.set_xlabel("Frames")
    ax.axvline(5, color="red", alpha=0.3, label="Mismatch onset")
    ax.axvline(15, color="green", alpha=0.3, label="End of response window")
    ax.axhline(0, color="green", alpha=0.3)

    ax.legend()

    return fig, ax

-  Activity is high before a mismatch
-  Current analysis selects cells that were already more active before the  mismatch.

## A challenge because you're dumb: find trials. 

I need to find six meters, then three seconds, then six meters.   

In [ ]:
closed_loop

In [ ]:

closed_loop = mismatch.find_trials(closed_loop)

In [ ]:

start= 0

end = -1


plt.plot(closed_loop["mouse_z"][start:end],closed_loop["trial_indicator"][start:end])

plt.plot(closed_loop["mouse_z"][start:end], closed_loop["mismatch"][start:end]*30)

In [ ]:
sync_loop, fig, ax = mismatch.check_trials(flexilims_session, recording, closed_loop)

### Verifying

In [ ]:
def get_mismatch_debug_file(flexilims_session, recording):

    raw = flz.get_data_root("raw", flexilims_session=flexilims_session)

    ds = flz.get_datasets(
        flexilims_session=flexilims_session,
        origin_name=recording.name,
        dataset_type="harp",
        allow_multiple=False,
    )

    filename=ds.csv_files["DebugMismatchDis"]

    MismatchDebug =pd.read_csv(raw/recording.path/filename)
    
    return  MismatchDebug



In [ ]:
def synchronize_dataframes(df_a, df_b):
    df_a['IsMismatch'] = 0 

    for i, (idx, row) in enumerate(df_a.iterrows()):
        value_a = row["mismatch_mouse_z"]*100
        
        # Find the row in df_b where the value in 'variable1' is closest to value_a
        closest_row = df_b.iloc[(df_b["MismatchDistance"] - value_a).abs().argmin()]
        #print((closest_row["MismatchDistance"], value_a))
        
        # Combine the rows
        df_a["IsMismatch"][i] = closest_row["IsMismatch"]
    
    return df_a

sync_loop = synchronize_dataframes(closed_loop, MismatchDebug)

In [ ]:

def plot_synchronized_data(closed_loop, sync_loop, start, end):
    fig, ax = plt.subplots(figsize=(20, 10))  # Very large figure
    
    ax.plot(closed_loop["mouse_z"][start:end], closed_loop["trial_indicator"][start:end], label="Trial indicator")
    ax.plot(closed_loop["mouse_z"][start:end], closed_loop["mismatch"][start:end]*30, label="Mismatch")
    ax.plot(closed_loop["mouse_z"][start:end], sync_loop["IsMismatch"][start:end]*30, label="Old mismatch window")
    
    ax.legend()
    ax.set_xlabel("Mouse Z")
    ax.set_ylabel("Values")
    ax.set_title("Synchronized Data Plot")
    
    plt.show()
    return fig, ax

# Example usage
# Assuming you have your dataframes closed_loop and sync_loop
# closed_loop = pd.DataFrame({...})
# sync_loop = synchronize_dataframes(closed_loop, other_dataframe, 'variable1')

# Plotting the data
start = 0
end = -1
fig, ax = plot_synchronized_data(closed_loop, sync_loop, start, end)

It's quite good

# Establish significance of the mismatch tuning

## How Attinger does it



The details of the statistical analysis are noted in the figure legends. We did not test the distribution of the data for normality. 

### What are the traces that they plot?

To quantify average response traces, we first calculated the average event-triggered fluorescence trace for each neuron. 

The responses of all neurons were then averaged and the baseline (mean DF/F in a 0.5 s window pre event onset) was subtracted. 


> *When does this happen?* 

> To quantify the significance of the difference of two average calcium responses as a function of time, we performed a separate Student’s t test for every bin of the calcium trace (10 Hz or 15 Hz) and marked bins as significantly different for p < 0.01. For visual clarity, we removed isolated significant bins, such that a significant bin was only marked if at least one of the two neighboring bins was also significant. 

### When is a neuron significantly modulated by mismatch?

To calculate the average response of each neuron to mismatch or playback halt, we first calculated the difference between the average event-triggered response and the average response to 1000 randomly triggered events to generate a random-corrected trace. Average responses to mismatch and playback halt were then calculated as the mean fluorescence of the random-corrected average in a response window minus the mean fluorescence in a baseline window for each neuron (the response window for mismatch, playback halt, running onset and playback onset was +500 ms to +1500 ms, and the baseline subtraction window was 1000 ms to 0 ms). 

To determine the significance of a neuron’s response, we calculated individual neuron responses to each mismatch event as described above and compared this distribution to the distribution generated by 1000 randomly triggered events. Significance was determined with a two sided Mann-Whitney-U test (p < 0.05). 

### Which mismatches count?

For mismatch and random events to be included in the analysis, mice had to be running above threshold (102 cm/s) before and after event onset (from 600 ms to + 1100 ms). 

In addition, for playback halt events to be included, mice had to be stationary during the playback halt (no running from 600 ms to +1100 ms). 

For running onset, mice had to be stationary for at least 600 ms prior to the running onset and continue running for 1100 ms above threshold following the onset. Similarly, for playback onset (quantified only during open-loop sessions) there had to be no visual flow for 600 ms prior to visual flow onset, followed by continuous visual flow above threshold for at least 1100 ms after onset, mice had to be stationary during this time.

### Cutoffs

In [ ]:
closed_loop

## How we do it

In [ ]:
closed_loop

1. Generate a set of randomly triggered events on suitable parts of the trial.
2. Is that distribution different from the mismatch distribution?

###  Generate  suitable  distribution


In [ ]:
def define_window_for_mismatch(closed_loop, corridor_length = 6):

    # Add a new column for the trial indicator
    closed_loop['mismatch_window'] = False
    closed_loop["in_trial"]  = np.where(closed_loop['trial_indicator'] > 0, True, False)

    # Define the thresholds
    beggining_threshold = corridor_length*(1/3)  
    end_threshold = corridor_length*(5/6)    

    # Initialize variables for trial tracking
    start_distance = 0
    n_rows = len(closed_loop)
    print(n_rows)
    i = 0


    while i < n_rows:

        current_distance = closed_loop["mouse_z"].iloc[i] - start_distance
        in_trial = closed_loop["in_trial"].iloc[i]

        if closed_loop["in_trial"].iloc[i] == 1 and closed_loop["in_trial"].iloc[i-1] == 0:
            start_distance = closed_loop["mouse_z"].iloc[i]

        # Assign mismatch window indicator
        while i < n_rows and current_distance >= beggining_threshold and current_distance < end_threshold:
            in_trial = closed_loop["in_trial"].iloc[i]

            if in_trial:
                closed_loop.loc[i, 'mismatch_window'] = True
            i += 1
            if i < n_rows:
                current_distance = closed_loop["mouse_z"].iloc[i] - start_distance
        
        i += 1
    

    return closed_loop



In [ ]:
fig,  ax =  mismatch.plot_synchronized_data(closed_loop, sync_loop)
ax.plot(closed_loop["mouse_z"], closed_loop["mismatch_window"]*30, label="mismatch window")
ax.legend()

plt.show()


In [ ]:
mis_closed_loop = closed_loop[closed_loop["mismatch_window"]]

In [ ]:
mis_closed_loop.to_csv("bananas.csv", index =  True)

In [ ]:
indices = mis_closed_loop.index.tolist()

In [ ]:
def generate_plausible_mismatch_indices(closed_loop):
    mis_closed_loop = closed_loop[closed_loop["mismatch_window"]]
    indices = mis_closed_loop.index.tolist()
    return indices

### Test distribution

In [ ]:
null_mode ="trial_structure"
save =True

print("Estimating mismatch distribution")
closed_loop = mismatch.find_mismatch(closed_loop)
closed_loop, idxs = mismatch.create_mismatch_window(
    closed_loop, window_start=5, window_end=20
)
neurons, neurons_df = mismatch.build_neurons_df(closed_loop)
misperneuron = mismatch.build_mismatches_per_neuron_list(
    neurons, neurons_df, window_start=5, window_end=20
)
mismatch_raster = mismatch.raster(neurons, misperneuron, window_start=5, window_end=20)

if null_mode == "trial_structure":

    closed_loop = mismatch.find_trials(closed_loop)
    closed_loop = mismatch.define_window_for_mismatch(closed_loop)
    indices = mismatch.generate_plausible_mismatch_indices(closed_loop)
else:
    indices = None

print("Estimating null distribution")
rand_raster, rand_misperneuron = mismatch.make_rand_raster(closed_loop, n_events=200, window_end=10, indices = indices)
sorted_mismatch_raster, modulation_raster = mismatch.modulation_sort_raster(rand_raster, mismatch_raster)
sorted_p = mismatch.calculate_significance(misperneuron, rand_misperneuron, modulation_raster)

print("Plotting")
rasterfig, rasterax = mismatch.plot_raster(sorted_mismatch_raster)
rasterfig, rasterax, rasterax2 =  mismatch.plot_significance(rasterfig, rasterax, sorted_p)
plt.show()
popfig, popax = mismatch.plot_pop_response(sorted_mismatch_raster)
plt.show()
sync_loop, fig, ax = mismatch.check_trials(flexilims_session, recording, closed_loop)

print("Saving...")

if save:

    ## save stuff

    processed = flz.get_data_root("processed", flexilims_session=flexilims_session)
    path = processed / recording.path

    #getting unsorted significance of modulation
    p = mismatch.calculate_significance(misperneuron, rand_misperneuron)

    #save dataframe
    mismatch_df =  {
        'modulation_size': modulation_raster,
        'p_value': p
    }
    mismatch_df = pd.DataFrame(mismatch_df)
    mismatch_df.to_pickle(str(path / "mismatch_df.pkl"))

    #save mismatch_raster
    np.save(str(path / "mismatch_raster.npy"), mismatch_raster)

    #save figures
    rasterfig.savefig(str(path / "raster"))
    popfig.savefig(str(path / "population"))

In [ ]:
recording.path

In [ ]:
processed = flz.get_data_root("processed", flexilims_session=flexilims_session)
processed

We must test `misperneuron` and `rand_misperneuron` 

In [ ]:
def calculate_significance(misperneuron, rand_misperneuron, modulation_raster = None):

    neuron_p = np.zeros(len(misperneuron))

    for neuron in tqdm(range(len(misperneuron))):
        mis_responses = misperneuron[neuron][:,5:15] #keep the rersponse part
        rand_responses  = rand_misperneuron[neuron]

        mis_mean = np.mean(mis_responses, axis=1)
        rand_mean =(np.mean(rand_responses, axis = 1))
        p = mannwhitneyu(mis_mean, rand_mean)
        neuron_p[neuron] = p.pvalue
    
    if modulation_raster is not None:

        sorted_indices = np.argsort(-modulation_raster)
        sorted_p = neuron_p[sorted_indices]

        return sorted_p
    
    else:

        return neuron_p





In [ ]:
sorted_p = calculate_significance(misperneuron, rand_misperneuron, modulation_raster)

In [ ]:
sorted_indices = np.argsort(-modulation_raster)
sorted_p = neuron_p[sorted_indices]
plt.plot(sorted_p)
plt.ylim((0, 0.05))

In [ ]:
plot_sorted_p = np.where(sorted_p<0.05, 1, 0)
plt.plot(plot_sorted_p)
plt.show()
plot_sorted_p = plot_sorted_p[np.newaxis,  :]
plt.imshow(plot_sorted_p.T, aspect = "auto")


In [ ]:

fig, ax = mismatch.plot_raster(sorted_mismatch_raster)
# The list [left, bottom, width, height] specifies the subplot's position and size in figure coordinates (0 to 1)

def add_significance(fig, ax, sorted_p):

    #Prepare for plotting
    plot_sorted_p = np.where(sorted_p<0.05, 1, 0)
    plot_sorted_p = plot_sorted_p[np.newaxis,  :]

    ax2 = fig.add_axes([0.06, 0.1, 0.03, 0.8], sharey = ax)  # Adjust these values as needed
    ax2.imshow(plot_sorted_p.T, aspect ="auto", cmap ="binary")
    ax.yaxis.set_visible(False)
    ax2.xaxis.set_visible(False)
    ax2.set_ylabel("Neurons", size =  25)

    return fig, ax, ax2


## Piping

For every recording, we want to save three plots, a dataframe that looks like Yiran's `neurons_df`, and a dataframe with the size of the mismatch responses and their significance. 

So far: when given a session, get the recordings, iterate through the sync dataframes, analyse, move to the next recording. 

In [ ]:
yirans = pd.read_pickle("/camp/lab/znamenskiyp/home/shared/projects/depth_mismatch_seq/BRAC9057.4j/S20240517/neurons_df.pickle")

In [ ]:
yirans

# Debug  :(

In [ ]:
session = "BRAC9057.4j_S20240517"
flexilims_session = None

In [ ]:
#  get the right stuff

if flexilims_session is None:
    flexilims_session = flz.get_flexilims_session(project_id=PROJECT)

exp_session = flz.get_entity(
    datatype="session", name=session, flexilims_session=flexilims_session
)

vs_df_all, imaging_df_all, recordings = mismatch.sync_all_recordings(
    session_name=session,
    flexilims_session=flexilims_session,
    project=PROJECT,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base=PROTOCOL,
    photodiode_protocol=5,
    return_volumes=True,
)

recordings = flz.get_entities(
    datatype="recording",
    origin_id=exp_session['id'],
    query_key="recording_type",
    query_value="two_photon",
    flexilims_session=flexilims_session,
)

recordings = recordings[recordings.name.str.contains(PROTOCOL)]

In [ ]:
i = 0
recname = recordings.name[0]

recording = flz.get_entity(
    datatype="recording",
    name=recname,
    flexilims_session=flexilims_session,
)



closed_loop = imaging_df_all[i]
is_playback = mismatch.determine_if_playback(recording, flexilims_session)


In [ ]:
print("Estimating mismatch distribution")
closed_loop = mismatch.find_mismatch(closed_loop, is_playback)
closed_loop, idxs = mismatch.create_mismatch_window(
    closed_loop, window_start=5, window_end=20
)
neurons, neurons_df = mismatch.build_neurons_df(closed_loop)

In [ ]:
beg= 2350
end = 2450

plt.plot(closed_loop["range_indicator"][beg:end], label  = "range_indicator")
plt.plot(closed_loop["mismatch"][beg:end], label = "mismatch")
plt.plot(closed_loop["mism_ratio"][beg:end])

#closed_loop = mismatch.find_trials(closed_loop)
plt.plot(closed_loop["trial_indicator"][beg:end]/20, label = "trial_indicator")


plt.legend()

plt.show()

plt.plot(closed_loop["mouse_z"][beg:end]/150, label = "mouse_z")
plt.plot((closed_loop["mismatch_mouse_z"][beg:end]+6)/150, label = "mismatch_mouse_z")

plt.legend()

plt.show()


In [ ]:
closed_loop["mouse_z"][1]

In [ ]:
beg= 2400
end = 2600
plt.plot(closed_loop["mouse_z"][beg:end])
plt.plot(closed_loop["mismatch"][beg:end]*150)

plt.plot(closed_loop["mismatch_mouse_z"][beg:end]+6)

In [ ]:
window_start = 5
window_end = 20
indices = None

mismatches_per_neuron = list(np.zeros(neurons))

window = window_start + window_end

neurons_df["start_mismatch"] = np.zeros(len(neurons_df["range_indicator"]))

neurons_df.loc[1:, "start_mismatch"] = np.diff(neurons_df["range_indicator"])

if indices is None:
    n_mismatches = len(
        neurons_df["start_mismatch"][neurons_df["start_mismatch"] == 1]
    )
else:
    n_mismatches = len(indices)

print(f"# mismatches: {n_mismatches}")

for i in range(neurons):
    mismatches_per_neuron[i] = np.zeros((n_mismatches, window))

# Initialize variables to track the start and end of intervals
in_interval = False
start_idx = None

if indices is None:
    # Iterate through the DataFrame to identify intervals
    idx_mismatch = -1
    print(f"Building {n_mismatches} mismatches per neuron")
    for idx, row in tqdm(neurons_df.iterrows()):
        if row["range_indicator"] == 1 and not in_interval:
            # Start of a new interval
            start_idx = idx
            in_interval = True
            idx_mismatch += 1
            # print(f"This is mismatch {idx_mismatch}")
        elif row["range_indicator"] == 0 and in_interval:
            # End of the current interval
            end_idx = idx - 1
            for neuron in range(neurons):
                mismatches_per_neuron[neuron][idx_mismatch, :] = neurons_df[
                    f"neuron{neuron}"
                ][start_idx:end_idx]
            in_interval = False
            # print(f"start and end idx: {(start_idx, end_idx)}")

else:
    print(f"Building {n_mismatches} mismatches per neuron")
    nframes = len(neurons_df)
    for idx_mismatch, idx in tqdm(enumerate(indices)):
        start_idx = max(0, idx - window_start)
        end_idx = min((nframes - 1), idx + window_end)
        for neuron in range(neurons):
            #You need to make sure the window always has the right size. 
            slice_data = neurons_df[f"neuron{neuron}"][start_idx:end_idx].values
            if len(slice_data) < window:
                slice_data = np.pad(slice_data, (0, window - len(slice_data)), 'constant')
            mismatches_per_neuron[neuron][idx_mismatch, :] = slice_data

In [ ]:
plt.plot(neurons_df["range_indicator"])

In [ ]:
plt.plot(closed_loop["mismatch"])

Okay. Of course it breaks. It's always uncoupled! I need to find playback halts. 

In [ ]:
ds = flz.get_datasets(
    flexilims_session=flexilims_session,
    origin_name=recording.name,
    dataset_type="harp",
    allow_multiple=False,
)

In [ ]:
closed_loop

#  Depth

Concatenate the dataframes adequately. Plot the correlation between depth and mism. response. 

In [ ]:
session = "BRAC9057.4j_S20240517"
flexilims_session = None

In [ ]:
depth_df = mismatch.build_depth_df(session)
depth_df = mismatch.process_depth_df(depth_df)

## How we find it

In [ ]:
session = "BRAC9057.4j_S20240517"
flexilims_session = None
#  get the right stuff

def build_depth_df(session, index =  0, flexilims_session =  None):
    
    if flexilims_session is None:
        flexilims_session = flz.get_flexilims_session(project_id=PROJECT)

    exp_session = flz.get_entity(
        datatype="session", name=session, flexilims_session=flexilims_session
    )

    recordings = flz.get_entities(
        datatype="recording",
        origin_id=exp_session['id'],
        query_key="recording_type",
        query_value="two_photon",
        flexilims_session=flexilims_session,
    )

    recordings = recordings[recordings.name.str.contains(PROTOCOL)]
    i = index
    recname = recordings.name[i]

    recording = flz.get_entity(
        datatype="recording",
        name=recname,
        flexilims_session=flexilims_session,
    )

    processed = flz.get_data_root("processed", flexilims_session=flexilims_session)
    yiran_path = processed / exp_session.path / "neurons_df.pickle"
    yiran_df = pd.read_pickle(yiran_path)

    mismatch_path  =  processed / recording.path / "mismatch_df.pkl"
    mismatch_df = pd.read_pickle(mismatch_path)

    depth_df =  {"is_depth_neuron": yiran_df["is_depth_neuron"], 
             "preferred_depth_closedloop_crossval": yiran_df["preferred_depth_closedloop_crossval"], 
             "depth_tuning_test_spearmanr_pval_closedloop": yiran_df["depth_tuning_test_spearmanr_pval_closedloop"], 
             "mis_modulation_size": mismatch_df["modulation_size"], 
             "mis_p_value": mismatch_df["p_value"]}
    
    depth_df = pd.DataFrame(depth_df)

    return depth_df


In [ ]:
recording.path

In [ ]:
#yirans = pd.read_pickle("/camp/lab/znamenskiyp/home/shared/projects/depth_mismatch_seq/BRAC9057.4j/S20240517/neurons_df.pickle")
processed = flz.get_data_root("processed", flexilims_session=flexilims_session)
yiran_path = processed / exp_session.path / "neurons_df.pickle"
yiran_df = pd.read_pickle(yiran_path)

In [ ]:
mismatch_path  =  processed / recording.path / "mismatch_df.pkl"
mismatch_df = pd.read_pickle(mismatch_path)

In [ ]:
depth_df =  {"is_depth_neuron": yiran_df["is_depth_neuron"], 
             "preferred_depth_closedloop_crossval": yiran_df["preferred_depth_closedloop_crossval"], 
             "depth_tuning_test_spearmanr_pval_closedloop": yiran_df["depth_tuning_test_spearmanr_pval_closedloop"], 
             "mis_modulation_size": mismatch_df["modulation_size"], 
             "mis_p_value": mismatch_df["p_value"]}
depth_df = pd.DataFrame(depth_df)

In [ ]:
log_preferred_depth = np.log10(depth_df["preferred_depth_closedloop_crossval"])
plt.scatter(np.log10(depth_df["preferred_depth_closedloop_crossval"]), depth_df["mis_modulation_size"], marker = ".")
plt.ylabel("mismatch modulation")
plt.xlabel("log preferred depth")


# Calculate correlation
corr_coef, p_value = pearsonr(log_preferred_depth, depth_df["mis_modulation_size"])

# Display correlation coefficient
plt.title(f"Correlation: {corr_coef:.2f} (p-value: {p_value:.2e})")

In [ ]:
#sig_depth_df = depth_df[depth_df["mis_p_value"]<0.05]
#sig_depth_df = sig_depth_df[sig_depth_df["is_depth_neuron"]]

def process_depth_df(depth_df, threshold = 0.05):
    sig_depth_df = depth_df[depth_df["mis_p_value"]<threshold]
    sig_depth_df = sig_depth_df[sig_depth_df["is_depth_neuron"]]
    positive_mismatch = sig_depth_df[sig_depth_df["mis_modulation_size"]>0]
    negative_mismatch = sig_depth_df[sig_depth_df["mis_modulation_size"]<0]

    return sig_depth_df, positive_mismatch, negative_mismatch

In [ ]:
def plot_correlation_with_fit(sig_depth_df):
    
    log_sig_preferred_depth = np.log10(sig_depth_df["preferred_depth_closedloop_crossval"])

    fig, ax = plt.subplots(facecolor =  "white")

    # Scatter plot
    ax.scatter(log_sig_preferred_depth, sig_depth_df["mis_modulation_size"], marker=".")
    ax.set_ylabel("Mismatch Modulation")
    ax.set_xlabel("Log Preferred Depth")

    # Calculate correlation
    corr_coef, p_value = pearsonr(log_sig_preferred_depth, sig_depth_df["mis_modulation_size"])

    # Linear fit
    fit = np.polyfit(log_sig_preferred_depth, sig_depth_df["mis_modulation_size"], 1)
    fit_fn = np.poly1d(fit)

    # Plot the best fit line
    ax.plot(log_sig_preferred_depth, fit_fn(log_sig_preferred_depth), color="red", label=f'Best fit line: y={fit[0]:.2f}x + {fit[1]:.2f}')

    # Display correlation coefficient
    ax.set_title(f"Correlation: {corr_coef:.2f} (p-value: {p_value:.2e})")
    ax.legend()

    return fig, ax

In [ ]:
positive_mismatch = sig_depth_df[sig_depth_df["mis_modulation_size"]>0]
negative_mismatch = sig_depth_df[sig_depth_df["mis_modulation_size"]<0]

## Pull them for three sessions

In [ ]:
flexilims_session = None

#14 is not processed?

sessionlist = ["BRAC9057.4j_S20240517", 
               "BRAC9057.4j_S20240516", 
               "BRYB177.4e_S20240520", 
               "BRYB177.4e_S20240521",
                "BRYB177.4e_S20240524", 
               "BRAC9057.4i_S20240430"
               ]
               #placeholder, ideally  it  chooses them automatically

depthlist = list(np.zeros(len(sessionlist)))

for i, session in tqdm(enumerate(sessionlist)):

    depth_df =  mismatch.build_depth_df(session, 0, flexilims_session)
    depth_df["origin"] = session
    depthlist[i] = depth_df

In [ ]:
complete_depth =  pd.concat(depthlist, ignore_index =   True)

In [ ]:
sig_depth_df, positive_mismatch, negative_mismatch = mismatch.process_depth_df(complete_depth)

In [ ]:
len(positive_mismatch)

In [ ]:
def plot_differential_histograms(positive_mismatch, negative_mismatch):

    fig, ax = plt.subplots(1, 2, sharex = True, facecolor = "white")
    ax[0].hist(np.log10(positive_mismatch["preferred_depth_closedloop_crossval"]), bins  =  100)
    ax[0].set_title(f"Positive mismatch, n={len(positive_mismatch)}")
    ax[0].set_xlim((-2, 2))
    ax[0].axvline(np.median(np.log10(positive_mismatch["preferred_depth_closedloop_crossval"])), color  = "red")


    ax[1].hist(np.log10(negative_mismatch["preferred_depth_closedloop_crossval"]), bins  =  100)
    ax[1].set_title(f"Negative mismatch,  n={len(negative_mismatch)}")
    ax[1].set_xlim((-2, 2))
    ax[1].axvline(np.median(np.log10(negative_mismatch["preferred_depth_closedloop_crossval"])), color = "red")

    ax[1].set_xlabel("log pref depth")
    ax[0].set_xlabel("log pref depth")
    ax[0].set_ylabel("Count")


    p =  mannwhitneyu(np.log10(positive_mismatch["preferred_depth_closedloop_crossval"]), np.log10(negative_mismatch["preferred_depth_closedloop_crossval"]))
    fig.suptitle(f"M-W U: p = {p.pvalue:.2e}")
    print(p)

    return  fig,  ax

In [ ]:
fig, ax = mismatch.plot_differential_histograms(positive_mismatch, negative_mismatch)
plt.show()

In [ ]:
fig, ax = mismatch.plot_correlation_with_fit(sig_depth_df)
plt.show()               

In [ ]:
fig, ax = mismatch.plot_correlation_with_fit(positive_mismatch)
plt.show()

In [ ]:
fig, ax = mismatch.plot_correlation_with_fit(negative_mismatch)
plt.show()

In [ ]:
# Define sets for the two conditions
# Define sets for the two conditions
depth = set(complete_depth.index[(complete_depth["depth_tuning_test_spearmanr_pval_closedloop"]<0.05) & (complete_depth["depth_tuning_test_spearmanr_rval_closedloop"]>0) ])
mismatch_set = set(complete_depth.index[complete_depth['mis_p_value'] < 0.05])
all_neurons = set(complete_depth.index)

# Create a figure and axis with a white background
fig, ax = plt.subplots(figsize=(8, 8))
fig.patch.set_facecolor('white')
ax.set_facecolor('white')

# Create a Venn diagram
venn3([depth, mismatch_set, all_neurons], ('Depth neurons', 'Mismatch neurons', "All neurons"), ax=ax)

# Set title
ax.set_title('Venn Diagram for Depth and Mismatch neurons')

# Show the plot
plt.show()

# Multigain analysis

Having a look

In [ ]:
session = "BRYB177.4e_S20240530"
flexilims_session = None
#  get the right stuff

if flexilims_session is None:
    flexilims_session = flz.get_flexilims_session(project_id=PROJECT)

exp_session = flz.get_entity(
    datatype="session", name=session, flexilims_session=flexilims_session
)

vs_df_all, imaging_df_all, recordings = mismatch.sync_all_recordings(
    session_name=session,
    flexilims_session=flexilims_session,
    project=PROJECT,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base=PROTOCOL,
    photodiode_protocol=5,
    return_volumes=True,
)

recordings = flz.get_entities(
    datatype="recording",
    origin_id=exp_session['id'],
    query_key="recording_type",
    query_value="two_photon",
    flexilims_session=flexilims_session,
)

recordings = recordings[recordings.name.str.contains(PROTOCOL)]
i = 2
recname = recordings.name[i]

recording = flz.get_entity(
    datatype="recording",
    name=recname,
    flexilims_session=flexilims_session,
)

closed_loop = imaging_df_all[i]
is_playback = mismatch.determine_if_playback(recording, flexilims_session)


## Building a pipeline

In [ ]:
plt.plot(closed_loop["mism_ratio"])

In [ ]:
# List of target values
target_values = [1, 0.25, 4]

# Function to find the closest value in the target list
def find_closest_value(value, target_values):
    return min(target_values, key=lambda x: abs(x - value))

# Apply the function to the series
filtered_series = closed_loop["mism_ratio"].apply(lambda x: find_closest_value(x, target_values))

#filter blips
closed_loop["filtered_mism_ratio"]  = filtered_series
for i in range(1, len(closed_loop)-1):
    if (closed_loop["filtered_mism_ratio"][i] != closed_loop["filtered_mism_ratio"][i-1]) and (closed_loop["filtered_mism_ratio"][i] != closed_loop["filtered_mism_ratio"][i+1]):
        closed_loop["filtered_mism_ratio"][i] = closed_loop["filtered_mism_ratio"][i-1]

In [ ]:
#plt.plot(closed_loop["mism_ratio"])
beg =  0
end = -1
plt.plot(closed_loop["filtered_mism_ratio"][beg:end], label =  "filtered", alpha  = 0.5)
#plt.plot(filtered_series[beg:end])
plt.plot(closed_loop["mism_ratio"][beg:end], label =  "unfiltered", alpha  = 0.5)
plt.legend()

Trials drift. What about blatantly usingg the debug tool to identify  trials the harder way? This  can help knowing what's a transition and what's a mismatch. 

In [ ]:
MismatchDebug = mismatch.get_mismatch_debug_file(flexilims_session, recording)

sync_loop = mismatch.synchronize_dataframes(closed_loop, MismatchDebug)

In [ ]:
sync_loop

In [ ]:
beg = 0
end = -1

plt.plot(sync_loop["mouse_z"][beg:end])
plt.plot(sync_loop["mismatch_mouse_z"][beg:end])

In [ ]:
sync_loop["mismatch"] = 0
sync_loop["trial_end"] = 0
sync_loop["in_trial"] = 0

for i in range(1, len(sync_loop)-1):
    if (sync_loop["IsMismatch"][i-1] == False) and (sync_loop["IsMismatch"][i+1] == True):
        sync_loop.loc[i-3:i, "mismatch"] = 1
    if (sync_loop["IsMismatch"][i-1] == True) and (sync_loop["IsMismatch"][i+1] == False):
        sync_loop.at[i, "trial_end"] = 1
        trial_start = sync_loop["mouse_z"][i] - ((5/6)*6)
        trial_end = sync_loop["mouse_z"][i] + ((1/6)*6)
        closest_row = sync_loop.iloc[(sync_loop["mouse_z"] - trial_start).abs().argmin()]
        start_index = closest_row.name
        closest_row = sync_loop.iloc[(sync_loop["mouse_z"] - trial_end).abs().argmin()]
        end_index =  closest_row.name
        sync_loop.loc[start_index:end_index, "in_trial"] = 1

In [ ]:

# Define the range
beg = 2000
end = 3000

# Create a figure and axis
fig, ax = plt.subplots(figsize=(14, 8))

# Plot the data
ax.plot(closed_loop["mouse_z"][beg:end], closed_loop["filtered_mism_ratio"][beg:end], label="Filtered", alpha=0.5)
ax.plot(closed_loop["mouse_z"][beg:end], closed_loop["mism_ratio"][beg:end], label="Unfiltered", alpha=0.5)
ax.plot(closed_loop["mouse_z"][beg:end], sync_loop["mismatch"][beg:end], label="Mismatch", alpha=0.5)
ax.plot(closed_loop["mouse_z"][beg:end], sync_loop["IsMismatch"][beg:end], label="IsMismatch", alpha=0.5)
ax.plot(closed_loop["mouse_z"][beg:end], sync_loop["in_trial"][beg:end], label="In Trial", alpha=0.5)

# Add legend
ax.legend()

# Set title and labels
ax.set_title('Trial from index {} to {}'.format(beg, end))
ax.set_xlabel('Index')
ax.set_ylabel('Values')

# Set white background
fig.patch.set_facecolor('white')
ax.set_facecolor('white')

# Show the plot
plt.show()



And now we find the window on which mismatches are possible

In [ ]:
closed_loop["trial_indicator"]=sync_loop["in_trial"]
closed_loop["estimated_mismatch"] =  sync_loop["mismatch"]
closed_loop = mismatch.define_window_for_mismatch(closed_loop)

In [ ]:
# Define the range
beg = 2000
end = 3000

# Create a figure and axis
fig, ax = plt.subplots(figsize=(14, 8))

# Plot the data
ax.plot(closed_loop["mouse_z"][beg:end], closed_loop["filtered_mism_ratio"][beg:end], label="Filtered", alpha=0.5)
ax.plot(closed_loop["mouse_z"][beg:end], closed_loop["mism_ratio"][beg:end], label="Unfiltered", alpha=0.5)
ax.plot(closed_loop["mouse_z"][beg:end], sync_loop["mismatch"][beg:end], label="Mismatch", alpha=0.5)
#ax.plot(closed_loop["mouse_z"][beg:end], sync_loop["IsMismatch"][beg:end], label="IsMismatch", alpha=0.5)
ax.plot(closed_loop["mouse_z"][beg:end], sync_loop["in_trial"][beg:end], label="In Trial", alpha=0.5)
ax.plot(closed_loop["mouse_z"][beg:end], closed_loop["mismatch_window"][beg:end], label="Mismatch window", alpha=0.5)


# Add legend
ax.legend()

# Set title and labels
ax.set_title('Trial from index {} to {}'.format(beg, end))
ax.set_xlabel('Mouse position')
ax.set_ylabel('Values')

# Set white background
fig.patch.set_facecolor('white')
ax.set_facecolor('white')

# Show the plot
plt.show()

## Running pipeline

In [ ]:
from cottage_analysis.analysis import mismatch

In [ ]:
fig, ax, closed_loop, gains_df =  mismatch.find_multimismatch(closed_loop, recording)

In [ ]:
closed_loop, idxs = mismatch.create_mismatch_window(
    closed_loop, window_start=5, window_end=20
)
neurons, neurons_df = mismatch.build_neurons_df(closed_loop)
misperneuron = mismatch.build_mismatches_per_neuron_list(
    neurons, neurons_df, window_start=5, window_end=20
)

In [ ]:


beg = 1400
end= 1600
#plt.plot(closed_loop["range_indicator"][beg:end])
plt.plot(closed_loop["in_trial"][beg:end], label = "in_trial")
plt.plot(closed_loop["trial_start"][beg:end], label ="trial_start")
#plt.plot(closed_loop["mismatch"][beg:end], label = "mismatch")
#plt.plot(closed_loop["filtered_mism_ratio"][beg:end])
#plt.plot(closed_loop["mism_ratio"][beg:end])
#plt.plot(closed_loop["mism_ratio_filled"][beg:end])

#print(closed_loop["mism_ratio_filled"][beg:end])
plt.legend()


In [ ]:
misperneuron[3].shape

In [ ]:
gains_df

In [ ]:

tt_misperneuron =  {
    "htm":[],
    "htl": [], 
    "mtl": [],
    "mth": [],
    "ltm": [],
    "lth": []
    }

# Dictionary to hold the indexes for each trial type
tt_indices = {
    "htm": [],
    "htl": [], 
    "mtl": [],
    "mth": [],
    "ltm": [],
    "lth": []
    }

# Identifying indexes for each trial type
tt_indices["htm"] = gains_df[(gains_df["start"] == 4) & (gains_df["end"] == 1)].index.tolist()
tt_indices["htl"] = gains_df[(gains_df["start"] == 4) & (gains_df["end"] == 0.25)].index.tolist()
tt_indices["mtl"] = gains_df[(gains_df["start"] == 1) & (gains_df["end"] == 0.25)].index.tolist()
tt_indices["mth"] = gains_df[(gains_df["start"] == 1) & (gains_df["end"] == 4)].index.tolist()
tt_indices["ltm"] = gains_df[(gains_df["start"] == 0.25) & (gains_df["end"] == 1)].index.tolist()
tt_indices["lth"] = gains_df[(gains_df["start"] == 0.25) & (gains_df["end"] == 4)].index.tolist()

# Mapping misperneuron values to tt_misperneuron based on tt_indices
for trial_type in tt_misperneuron.keys():
    tt_misperneuron[trial_type] = [misperneuron[idx] for idx in tt_indices[trial_type]]

In [ ]:
# Calculate np.diff and pad the result
trial_diff = np.diff(closed_loop["in_trial"])
padded_diff = np.insert(trial_diff, 0, 0)  # Insert 0 at the beginning to match the length
# Assign the padded_diff to the appropriate slice
closed_loop.loc[:, "trial_in_out"] = padded_diff

# Find the indices where trial_in_out is 1
beginnings = closed_loop[closed_loop["trial_in_out"] == 1].index
ends = closed_loop[closed_loop["trial_in_out"] == -1].index


# Initialize trial_gains array with the correct length
trial_gains = np.zeros(len(beginnings))

# Iterate through the indices and perform the desired operation
for index, trial in enumerate(beginnings):
    trial_gains[index] = closed_loop["filtered_mism_ratio"][trial+20]



In [ ]:
# Gain values
tt_gains = {
    "h": 4,
    "m": 1, 
    "l": 0.25,
}

# New dictionary to store filtered DataFrames
null = {}


for gain in tt_gains.keys():
    # Filter for moments with the correct gain
    trials = closed_loop[(closed_loop["filtered_mism_ratio"] == tt_gains[gain]) &
                         (closed_loop["range_indicator"] == 0)]
    null[gain] = trials


for gain in null.keys():
    plt.figure()
    plt.title(f"Filtered Trials for Gain {gain}")
    plt.plot(null[gain].index, null[gain]["filtered_mism_ratio"], '.')
    plt.xlabel("Index")
    plt.ylabel("Filtered Mism Ratio")
    plt.show()

In [ ]:
plt.scatter(range(len(filtered_trials["l"]["imaging_frame"])), filtered_trials["l"]["imaging_frame"])

In [ ]:
tt_null = {
    "htm": filtered_trials["h"],
    "htl": filtered_trials["h"], 
    "mtl": filtered_trials["m"],
    "mth": filtered_trials["m"],
    "ltm": filtered_trials["l"],
    "lth": filtered_trials["l"]
    }

In [ ]:
def break_into_trial_types(misperneuron, gains_df):

    tt_misperneuron =  {
        "htm":[],
        "htl": [], 
        "mtl": [],
        "mth": [],
        "ltm": [],
        "lth": []
        }

    # Dictionary to hold the indexes for each trial type
    tt_indices = {
        "htm": [],
        "htl": [], 
        "mtl": [],
        "mth": [],
        "ltm": [],
        "lth": []
        }

    # Identifying indexes for each trial type
    tt_indices["htm"] = gains_df[(gains_df["start"] == 4) & (gains_df["end"] == 1)].index.tolist()
    tt_indices["htl"] = gains_df[(gains_df["start"] == 4) & (gains_df["end"] == 0.25)].index.tolist()
    tt_indices["mtl"] = gains_df[(gains_df["start"] == 1) & (gains_df["end"] == 0.25)].index.tolist()
    tt_indices["mth"] = gains_df[(gains_df["start"] == 1) & (gains_df["end"] == 4)].index.tolist()
    tt_indices["ltm"] = gains_df[(gains_df["start"] == 0.25) & (gains_df["end"] == 1)].index.tolist()
    tt_indices["lth"] = gains_df[(gains_df["start"] == 0.25) & (gains_df["end"] == 4)].index.tolist()

    # Mapping misperneuron values to tt_misperneuron based on tt_indices
    for trial_type in tt_misperneuron.keys():
        tt_misperneuron[trial_type] = [misperneuron[idx] for idx in tt_indices[trial_type]]

    return tt_misperneuron

def build_null_dist_per_trial_type(closed_loop):
    # Gain values
    tt_gains = {
        "h": 4,
        "m": 1, 
        "l": 0.25,
    }

    # New dictionary to store filtered DataFrames
    null = {}

    for gain in tt_gains.keys():
        # Filter for moments with the correct gain
        trials = closed_loop[(closed_loop["filtered_mism_ratio"] == tt_gains[gain]) &
                            (closed_loop["range_indicator"] == 0)]
        null[gain] = trials.index.tolist()


    for gain in null.keys():
        plt.figure()
        plt.title(f"Filtered Trials for Gain {gain}")
        plt.plot(null[gain], closed_loop["filtered_mism_ratio"][null[gain]], '.')
        plt.xlabel("Index")
        plt.ylabel("Filtered Mism Ratio")
        plt.show()
    
    return null

In [ ]:
def make_tt_rasters(tt_misperneuron, null, neurons):

    tt_raster = {}
    tt_rand_raster = {}
    tt_rand_misperneuron = {}

    for gain in tt_misperneuron.keys():
        tt_raster[gain] = raster(neurons, tt_misperneuron[gain], window_start=5, window_end=20)
        tt_rand_raster[gain], tt_rand_misperneuron[gain] = make_rand_raster(closed_loop, n_events=200, window_end=10, indices=null[gain])

    return tt_raster, tt_rand_raster, tt_rand_misperneuron

def make_tt_sorted_raster(tt_rand_raster, tt_raster):

    tt_sorted_mismatch_raster = {}
    tt_modulation_raster = {}

    for gain in tt_raster.keys():
        tt_sorted_mismatch_raster[gain], tt_modulation_raster[gain] = modulation_sort_raster(tt_rand_raster[gain], tt_raster[gain])
    
    return tt_sorted_mismatch_raster,  tt_modulation_raster

def calculate_tt_significance(tt_misperneuron, tt_rand_misperneuron, tt_modulation_raster):

    tt_sorted_p = {}

    for gain in tt_misperneuron.keys():
        tt_sorted_p[gain] = calculate_significance(tt_misperneuron[gain], tt_rand_misperneuron[gain], tt_modulation_raster[gain])
    
    return tt_sorted_p



## Plotting rasters

In [ ]:
tt_raster, tt_sorted_p = mismatch.analyse_multimismatch(closed_loop, recording)

In [ ]:
mismatch.plot_raster_grid(tt_raster, tt_sorted_p)

Okay, we debug

In [ ]:
findfig, findax, closed_loop, gains_df  = mismatch.find_multimismatch(closed_loop, recording,  is_playback, flexilims_session)

closed_loop, idxs = mismatch.create_mismatch_window(
    closed_loop, window_start=5, window_end=20
    )
neurons, neurons_df = mismatch.build_neurons_df(closed_loop)

misperneuron = mismatch.build_mismatches_per_neuron_list(
    neurons, neurons_df, window_start=5, window_end=20
    )

tt_misperneuron  = mismatch.break_into_trial_types(misperneuron, gains_df)

null = mismatch.build_null_dist_per_trial_type(closed_loop)

In [ ]:
tt_raster, tt_rand_raster, tt_rand_misperneuron = mismatch.make_tt_rasters(tt_misperneuron, null, neurons, closed_loop)


In [ ]:
sort = "all"

tt_sorted_mismatch_raster,  tt_modulation_raster = mismatch.make_tt_sorted_raster(tt_rand_raster, tt_raster, sort = sort)

tt_sorted_p = mismatch.calculate_tt_significance(tt_misperneuron, tt_rand_misperneuron, tt_modulation_raster, sort =sort)

In [ ]:
fig, axes = mismatch.plot_raster_grid(tt_sorted_mismatch_raster, tt_sorted_p)
fig.suptitle(f"Sorted {sort}", size  = 60)
plt.show()